In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -q torch==1.4.0 
!pip install transformers==3.5.1 --quiet

In [3]:
!pip install datasets
from datasets import load_dataset

In [4]:
!pip install pandas

In [5]:
label_dict={'Ratio of the decision' : 0 ,
            'Facts' : 1,
            'Precedent' : 2,
            'Argument' : 3,
            'Statute' : 4,
            'Ruling by Lower Court' : 5,
            'Ruling by Present Court' : 6
            }

In [6]:
import pandas as pd
import os
folder = '/content/drive/MyDrive/Aila2021/Task1'
df = None
for f in os.listdir(folder):
    filepath=os.path.join(folder,f)
    # print(filepath)
    if df is None:
        df = pd.read_csv(filepath,sep='\t',header=None)
    else:
        new_df = pd.read_csv(filepath,sep='\t',header=None)
        df = pd.concat([df,new_df])

df.columns = ['sentence','label']
df['num_labels'] = df['label'].map(label_dict)
train_texts = df['sentence'].tolist()
train_labels = df['num_labels'].tolist()
train_temp=train_texts
train_label_temp=train_labels

from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.1)

In [7]:
train_small,_,tlabel_small,_=train_test_split(train_temp,train_label_temp,test_size=0.2)

In [8]:
import pandas as pd
import os
folder = '/content/drive/MyDrive/Aila2021/test-aila-task1-2021'

df = None
for f in os.listdir(folder):
    filepath=os.path.join(folder,f)
    print(filepath)
    if df is None:
        df = pd.read_csv(filepath,sep='\t',header=None)
    else:
        new_df = pd.read_csv(filepath,sep='\t',header=None)
        df = pd.concat([df,new_df])

df.columns = ['sentence','label']
df['num_labels'] = df['label'].map(label_dict)

test_texts = df['sentence'].tolist()
test_labels = df['num_labels'].tolist()

/content/drive/MyDrive/Aila2021/test-aila-task1-2021/d10.txt
/content/drive/MyDrive/Aila2021/test-aila-task1-2021/d6.txt
/content/drive/MyDrive/Aila2021/test-aila-task1-2021/d2.txt
/content/drive/MyDrive/Aila2021/test-aila-task1-2021/d5.txt
/content/drive/MyDrive/Aila2021/test-aila-task1-2021/d4.txt
/content/drive/MyDrive/Aila2021/test-aila-task1-2021/d8.txt
/content/drive/MyDrive/Aila2021/test-aila-task1-2021/d9.txt
/content/drive/MyDrive/Aila2021/test-aila-task1-2021/d7.txt
/content/drive/MyDrive/Aila2021/test-aila-task1-2021/d3.txt
/content/drive/MyDrive/Aila2021/test-aila-task1-2021/d1.txt


In [33]:
with open('/content/drive/MyDrive/Aila2021/SeqOutput_1/test_texts.txt','w') as f:
  for item in test_texts:
        f.write("%s\n" % item)

In [9]:
import pandas as pd
import os
folder = '/content/drive/MyDrive/Aila2021/test-2-aila-task1-2021'
list1=[]
for filepath in os.listdir(folder):
  with open(os.path.join(folder,filepath),'r') as f:
    for line in f:
      list1.append(line.strip())

true_test_texts=list1
true_test_labels=[0]*len(list1)

In [35]:
len(true_test_texts)

848

In [34]:
with open('/content/drive/MyDrive/Aila2021/SeqOutput_1/true_test_texts.txt','w') as f:
  for item in true_test_texts:
        f.write("%s\n" % item)

In [10]:
tokenizer_checkpoint = 'roberta-base'
model_checkpoint = '/content/drive/MyDrive/Aila2021/roberta-retrained-model/'

In [11]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint, use_fast=True)

In [12]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
true_test_encodings = tokenizer(true_test_texts, truncation=True, padding=True)
train_small_encodings=tokenizer(train_small,truncation=True,padding=True)

In [13]:
import torch

class Aila2021(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Aila2021(train_encodings, train_labels)
val_dataset = Aila2021(val_encodings, val_labels)
test_dataset = Aila2021(test_encodings, test_labels)
true_test_dataset = Aila2021(true_test_encodings,true_test_labels)
train_small=Aila2021(train_small_encodings,tlabel_small)

In [14]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 7
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [15]:
from datasets import load_metric
metric = load_metric('glue','mnli')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [16]:
# training_args = TrainingArguments(
#     output_dir='/content/drive/MyDrive/Aila2021/SeqOutput_1',          # output directory
#     num_train_epochs=3,              # total number of training epochs
#     per_device_train_batch_size=16,  # batch size per device during training
#     per_device_eval_batch_size=16,   # batch size for evaluation
#     learning_rate=2e-5,
#     weight_decay=0.01,               # strength of weight decay
#     save_steps=7000,
#     save_total_limit=1,
#     seed=1,
#     evaluation_strategy="epoch",     # evaluate each `logging_steps`
#     # save_strategy = 'no',
#     # fp16=True,                       # Use mixed precision
#     # fp16_opt_level="02", 
# )


# trainer = Trainer(
#     model_init=model_init,                         # the instantiated 🤗 Transformers model to be trained
#     args=training_args,                  # training arguments, defined above
#     train_dataset=train_small,         # training dataset
#     eval_dataset=val_dataset,            # evaluation dataset
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )

In [17]:
# !pip install optuna

In [18]:
# !pip install ray[tune]

In [19]:
import torch
torch.cuda.empty_cache()

import gc
# del variables
gc.collect()

1451

In [20]:
# import optuna
# from ray import tune
# best_run = trainer.hyperparameter_search(n_trials=4, direction="maximize") # ,backend="ray"

In [22]:
# trainer.train()

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Aila2021/SeqOutput_1',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    # warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    learning_rate=7.99218830200415e-06,
    seed = 34
)

trainer = Trainer(
    model_init=model_init,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
# for n, v in best_run.hyperparameters.items():
#     setattr(trainer.args, n, v)

trainer.train()

Some weights of the model checkpoint at /content/drive/MyDrive/Aila2021/roberta-retrained-model/ were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/Aila2021/roberta-retrained-model/ and are newly

Step,Training Loss
500,1.621271
1000,1.530988


TrainOutput(global_step=1359, training_loss=1.5480491744849154)

In [23]:
trainer.evaluate()

{'epoch': 1.0,
 'eval_accuracy': 0.22682119205298013,
 'eval_loss': 1.435961365699768}

In [24]:
trainer.save_model("/content/drive/MyDrive/Aila2021/SeqClass_1/")

In [25]:
test_results = trainer.predict(test_dataset)
true_test_results = trainer.predict(true_test_dataset)

In [26]:
# from transformers import AutoTokenizer, AutoModel, TrainingArguments, Trainer, AutoModelForSequenceClassification

# # tokenizer = AutoTokenizer.from_pretrained("roberta-base")
# model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Aila2021/SeqClass/")

# # arguments for Trainer
# test_args = TrainingArguments(
#     output_dir = '/content/drive/MyDrive/Aila2021/SeqOutput',
#     do_train = False,
#     do_predict = True,
#     per_device_eval_batch_size = 16,   
#     dataloader_drop_last = False    
# )

# # init trainer
# trainer_1 = Trainer(
#               model = model, 
#               args = test_args, 
#               compute_metrics = compute_metrics)

# test_results = trainer_1.predict(test_dataset)

In [27]:
test_results

PredictionOutput(predictions=array([[ 0.97945493,  0.30782604,  0.67981094, ...,  0.39302614,
        -0.8795814 , -1.2264819 ],
       [ 0.14610805,  0.88923687,  1.6217165 , ..., -0.5845326 ,
        -0.8794017 , -1.3730451 ],
       [ 0.22410238,  1.1819943 ,  1.4504354 , ..., -0.81560785,
        -0.8645751 , -1.271927  ],
       ...,
       [ 1.5865952 ,  0.48811942,  0.29672748, ..., -0.54756814,
        -0.84047085, -0.98211974],
       [ 1.9489042 , -0.32709432,  0.66514474, ..., -0.97274524,
        -0.72738606, -0.8881453 ],
       [ 0.64404   ,  0.86012447,  0.42218664, ..., -0.19712497,
        -0.91729736, -0.23053189]], dtype=float32), label_ids=array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 5,
       1, 3, 3, 3, 1, 3, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 3, 0, 0, 0, 6, 6, 6, 1, 1, 5, 

In [29]:
output_pred_file = os.path.join(training_args.output_dir, "label_id_test.txt")
with open(output_pred_file, "w") as writer:
  for pred in test_results.label_ids:
    writer.write("label_ids = '%s'\n" % pred)

output_pred_file = os.path.join(training_args.output_dir, "pred_results_test.txt")
with open(output_pred_file, "w") as writer:
  for pred in test_results.predictions:
      writer.write("predictions = %s\n" % pred)

In [31]:
true_test_pred_file = os.path.join(training_args.output_dir, "true_test_results.txt")
with open(output_pred_file, "w") as writer:
  for pred in test_results.predictions:
      writer.write("predictions = %s\n" % pred)